In [13]:
from rasterstats import zonal_stats
from pathlib import Path
import pandas as pd
import geopandas as gpd
import numpy as np
DATA_FOLDER = Path('C:/Users/shafner/slum_extent_mapping/data/population_disaggregation')

In [14]:
original_boundary_file = DATA_FOLDER / 'Admin_PopdataKigali' / 'census_data_kigali_utm.shp'
# making a copy of the original file
boundary_file = DATA_FOLDER / 'Admin_PopdataKigali' / 'sdg_data_kigali_utm.shp'
gdf = gpd.read_file(original_boundary_file)
gdf.to_file(boundary_file)

psi_2016_file = DATA_FOLDER / 'psi_2016.tif'
psi_2020_file = DATA_FOLDER / 'psi_2020.tif'

def add_metric_to_file(boundary_file: Path, raster_file: Path, metric: str, field_name: str, primary_key: str):
    assert(boundary_file.exists() and raster_file.exists())
    
    gdf = gpd.read_file(boundary_file)
    
    def feature2stats(f):
        properties = f['properties']
        key = properties[primary_key]
        value = properties[metric]
        return (key, value)
    
    features = zonal_stats(str(boundary_file), str(raster_file), layer=0, geojson_out=True)
    stats = [feature2stats(f) for f in features]
    keys = [stat[0] for stat in stats]
    values = [stat[1] for stat in stats]
    d = {primary_key: keys, field_name: values}
    df = pd.DataFrame(data=d)
    df = df.set_index(primary_key)
    gdf = gdf.join(df, on=primary_key)
    
    gdf.to_file(boundary_file)
    
gdf.head()

,District,Cell_ID,Sector_ID,Distr_ID,Prov_ID,Name,Cell,Sector,Province,VillageID,District_1,Sector_1,Cell_1,Village,Pop2016,Pop2020,Village_ID,PK,geometry
0,Nyarugenge,110101,1101,11,1,Gihanga,Akabahizi,Gitega,Kigali Town/Umujyi wa Kigali,NY001,NYARUGENGE,Gitega,Akabahizi,Gihanga,663.0,691.0,NY001,1,"POLYGON Z ((172141.421 9784352.504 0.000, 1721..."
1,Nyarugenge,110101,1101,11,1,Iterambere,Akabahizi,Gitega,Kigali Town/Umujyi wa Kigali,NY002,NYARUGENGE,Gitega,Akabahizi,Iterambere,517.0,528.0,NY002,2,"POLYGON Z ((172302.644 9784712.949 0.000, 1722..."
2,Nyarugenge,110101,1101,11,1,Izuba,Akabahizi,Gitega,Kigali Town/Umujyi wa Kigali,NY003,NYARUGENGE,Gitega,Akabahizi,Izuba,727.0,724.0,NY003,3,"POLYGON Z ((171966.311 9784712.260 0.000, 1719..."
3,Nyarugenge,110101,1101,11,1,Nyaburanga,Akabahizi,Gitega,Kigali Town/Umujyi wa Kigali,NY004,NYARUGENGE,Gitega,Akabahizi,Nyaburanga,218.0,236.0,NY004,4,"POLYGON Z ((172322.871 9784535.419 0.000, 1723..."
4,Nyarugenge,110101,1101,11,1,Nyenyeri,Akabahizi,Gitega,Kigali Town/Umujyi wa Kigali,NY005,NYARUGENGE,Gitega,Akabahizi,Nyenyeri,786.0,823.0,NY005,5,"POLYGON Z ((172022.687 9784476.849 0.000, 1720..."


In [15]:
add_metric_to_file(boundary_file, psi_2016_file, 'mean', 'mean 2016', 'PK')
add_metric_to_file(boundary_file, psi_2020_file, 'mean', 'mean 2020', 'PK')

C:\Users\shafner\AppData\Local\Continuum\anaconda3\envs\rasterstats\lib\site-packages\rasterstats\io.py:301: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


In [16]:
def add_land_consumption_rate(boundary_file: Path, psi_t1_field: str, psi_t2_field: str, dt: int):
    gdf = gpd.read_file(boundary_file)
    psi_t1 = gdf[psi_t1_field]
    psi_t2 = gdf[psi_t2_field]
    lcr = np.log(psi_t2 / psi_t1) / dt
    gdf['lcr'] = lcr
    gdf.to_file(boundary_file)
    
    
add_land_consumption_rate(boundary_file, 'mean 2016', 'mean 2020', 4)

In [17]:
def add_population_growth_rate(boundary_file: Path, pop_t1_field: str, pop_t2_field: str, dt: int):
    gdf = gpd.read_file(boundary_file)
    pop_t1 = gdf[pop_t1_field]
    pop_t2 = gdf[pop_t2_field]
    pgr = np.log(pop_t2 / pop_t1) / dt
    gdf['pgr'] = pgr
    gdf.to_file(boundary_file)
    
    
add_population_growth_rate(boundary_file, 'Pop2016', 'Pop2020', 4)

In [18]:
def add_lcrpgr(boundary_file: Path, lcr_field: str, pgr_field: str):
    gdf = gpd.read_file(boundary_file)
    lcr = gdf[lcr_field]
    pgr = gdf[pgr_field]
    lcrpgr = lcr / pgr
    gdf['lcrpgr'] = lcrpgr
    gdf.to_file(boundary_file)
    
    
add_lcrpgr(boundary_file, 'lcr', 'pgr')